# About Florida Insurance Analysis

Book combines historical publicly reported data from the FL department of insurance to evaluate trends in market share, claims frequency/severity, and hurricane losses.

Data downloaded from https://apps.fldfs.com/QSRNG/Reports/ReportCriteria.aspx on 04 and 05 April 2025.

Environment AKStandard (Python 3.11.9)

# Imports

In [112]:
import pandas as pd # data ingestion, manipulation
from os import listdir # importing raw data

# Data Extraction/Ingestion

Outline/Plan

function - import, remove grand total\
function - check/remove blank columns\
compile to a single df\
\
future: move to AW S3 bucket, SQL access

In [113]:
# listdir('./data/raw')[-1]

In [114]:
info = pd.read_excel('./data/raw/2009Q1.xlsx', skiprows = 14)
print(f"The imported shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}\n")
grand_total_row = info.shape[0] - 1
info.drop(grand_total_row, axis=0, inplace=True)
print(f"The updated shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}")

The imported shape is: (214, 65)
The last named insurers are:
212    TESLA PROPERTY & CASUALTY, INC.
213                                NaN
Name: Insurer Name, dtype: object

The updated shape is: (213, 65)
The last named insurers are:
211    STATE NATIONAL INSURANCE COMPANY INC.
212          TESLA PROPERTY & CASUALTY, INC.
Name: Insurer Name, dtype: object


In [120]:
info.columns

Index(['Rank', 'Insurer Name', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Policies In Force',
       'Unnamed: 9', 'Unnamed: 10',
       'Total $ value of exposure for policies in force that include wind coverage',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Number of policies canceled', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Number of policies nonrenewed', 'Unnamed: 21',
       'Number of policies canceled due to hurricane risk', 'Unnamed: 23',
       'Number of policies nonrenewed due to hurricane risk', 'Unnamed: 25',
       'Number of new policies written', 'Unnamed: 27',
       'Policies in force that exclude wind coverage', 'Unnamed: 29',
       'Total premiums written', 'Unnamed: 31',
       'Policies in force that include wind coverage', 'Unnamed: 33',
       'Total $ value of exposure for policies in force that exclude wind coverage',
       'Unnamed: 35',
       'Direct premium writt